### Extract functions from LL files

In [1]:
import CodePreprocessing as preprocessing
import json

In [2]:

# ----------------- user functions  --------------------
preprocessing.functions_preprocessing( llvm_file='aymon.ll', json_file='UserCode' )
with open( 'UserCode.json', 'r' ) as f:
    user_code = json.load( f ) 

# ----------------- Vulnerable function ------------------
preprocessing.functions_preprocessing(llvm_file='vulnFunc.ll', json_file='VulnerableCode' )
with open( 'VulnerableCode.json', 'r' ) as f:
    vulnerable_code = json.load( f )

vulnerable_function=[]
for key in vulnerable_code:
    vulnerable_function.append(vulnerable_code[key])


### Matching

In [3]:
import matchers as matcher
import LLNormalizer as normalizer
import Winnowing

In [4]:
def check_function_vulnerable(threshold,score1,score2,score3):
    return score1 > threshold and score2 > threshold and score3 > threshold

In [5]:
threshold=0.7
k=3
code_scores=dict()

for key in user_code:
    fn=user_code[key] 
    #fn=normalizer.NormalizeLLVM(fn)

    jaro_winkler=matcher.jaro_winkler_similarity(fn, vulnerable_function[0])
    levenshtein=matcher.levenshtein_similarity(fn, vulnerable_function[0])
    ratcliff_obershelp=matcher.ratcliff_obershelp_similarity(fn, vulnerable_function[0])
    trigram=matcher.trigram_similarity(fn, vulnerable_function[0])
    sorensen_dice=matcher.sorensen_dice_similarity(fn, vulnerable_function[0])
    jaccard_distance=matcher.jaccard_distance(fn, vulnerable_function[0])

    scores=[jaro_winkler,levenshtein,ratcliff_obershelp,trigram,sorensen_dice,jaccard_distance]
    scores.sort(reverse=True) 
    code_scores[key]=scores

with open('code_scores.json', 'w') as f:
    f.write(json.dumps(code_scores, indent=6))

In [6]:
normalizedvuln = normalizer.NormalizeLLVM(vulnerable_function[0])
for key in code_scores:
    if check_function_vulnerable(threshold,code_scores[key][0],code_scores[key][1],code_scores[key][2]):
        fn = user_code[key]
        normalizedfn = normalizer.NormalizeLLVM(fn)
        MOSS_Acc_metric1, MOSS_Acc_metric2, hits, misses1, misses2 = Winnowing.diff(normalizedfn, normalizedvuln, K= 50, WindowSize= 10, P= 10)
        print("Vulnerable function found:",key)
        if(MOSS_Acc_metric1>0.7 or MOSS_Acc_metric2>0.7):
            print(f"MOSS Caught this <3 !!!")
            print(f"Accuracy_Metric 1 = {MOSS_Acc_metric1}  ||  Accuracy_Metric 2 = {MOSS_Acc_metric2}\nhits: {hits} , misses: {misses1}, misses2: {misses2}")
        print('-----------------------')

Vulnerable function found: define void @bad() local_unnamed_addr {

MOSS Caught this <3 !!!
Accuracy_Metric 1 = 0.830739299610895  ||  Accuracy_Metric 2 = 0.5324189526184538
hits: 4270 , misses: 870, misses2: 3750
-----------------------
Vulnerable function found: define void @good() local_unnamed_addr {

MOSS Caught this <3 !!!
Accuracy_Metric 1 = 0.7728520988622989  ||  Accuracy_Metric 2 = 0.5604551920341394
hits: 3940 , misses: 1158, misses2: 3090
-----------------------


In [7]:
'''

'''

'\n\n'